# PC Analysis Summary

This notebook contains analyses related to Principal Component (PC) adjustment:

1. **PRS Distribution Across Ancestries** - Show how PRS varies by ancestry
2. **Refit Phi vs Original Phi** - Compare signature-disease associations before/after PC adjustment
3. **Original Lambda vs Load Lambda** - Compare patient-signature loadings
4. **Population Boost Comparison** - Compare which populations show most changes with PC adjustment

## Key Questions:
- Does PC adjustment change biological interpretations?
- Are signature-disease associations (phi) stable?
- Are patient signatures (lambda/theta) stable?
- Which populations are most affected by PC adjustment?


## 3. Original Lambda vs Load Lambda (Patient-Signature Loadings)


## 4. Population Boost Comparison (Which populations show most changes?)


In [ ]:
# Compare which populations show most changes with PC adjustment
if thetas_nopcs is not None and thetas_withpcs is not None and len(ancestry_array) > 0:
    print("="*80)
    print("POPULATION BOOST COMPARISON")
    print("="*80)
    
    # Calculate differences in theta by ancestry
    # Use same subset
    n_common = min(len(thetas_nopcs), len(thetas_withpcs), len(ancestry_array))
    thetas_nopcs_subset = thetas_nopcs[:n_common]
    thetas_withpcs_subset = thetas_withpcs[:n_common]
    ancestry_subset = ancestry_array[:n_common]
    
    # Calculate mean absolute change per patient
    # Average across signatures and timepoints
    theta_diff = np.abs(thetas_withpcs_subset - thetas_nopcs_subset)
    mean_change_per_patient = np.mean(theta_diff, axis=(1, 2))  # (N,)
    
    # Group by ancestry
    ancestry_changes = {}
    for anc in np.unique(ancestry_subset):
        if anc == 'UNK':
            continue
        mask = ancestry_subset == anc
        if mask.sum() > 100:  # Only include ancestries with sufficient samples
            ancestry_changes[anc] = {
                'mean_change': np.mean(mean_change_per_patient[mask]),
                'std_change': np.std(mean_change_per_patient[mask]),
                'n': mask.sum()
            }
    
    # Sort by mean change
    sorted_ancestries = sorted(ancestry_changes.items(), 
                              key=lambda x: x[1]['mean_change'], 
                              reverse=True)
    
    print("\nPopulation changes (sorted by mean absolute change):")
    print(f"{'Ancestry':<20} {'N':<10} {'Mean Change':<15} {'Std Change':<15}")
    print("-" * 70)
    
    for anc, stats in sorted_ancestries[:10]:
        print(f"{anc:<20} {stats['n']:<10} {stats['mean_change']:<15.6f} {stats['std_change']:<15.6f}")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left: Bar plot of mean changes
    ax1 = axes[0]
    anc_names = [anc for anc, _ in sorted_ancestries[:10]]
    mean_changes = [stats['mean_change'] for _, stats in sorted_ancestries[:10]]
    n_samples = [stats['n'] for _, stats in sorted_ancestries[:10]]
    
    bars = ax1.barh(range(len(anc_names)), mean_changes, alpha=0.7)
    ax1.set_yticks(range(len(anc_names)))
    ax1.set_yticklabels(anc_names, fontsize=10)
    ax1.set_xlabel('Mean Absolute Change in Theta', fontsize=12, fontweight='bold')
    ax1.set_title('Population Changes with PC Adjustment\n(Top 10 by Mean Change)', 
                 fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Add sample size labels
    for i, (bar, n) in enumerate(zip(bars, n_samples)):
        ax1.text(bar.get_width(), i, f'  n={n:,}', 
                va='center', fontsize=9)
    
    # Right: Box plot of changes
    ax2 = axes[1]
    change_data = []
    change_labels = []
    for anc, stats in sorted_ancestries[:5]:  # Top 5
        mask = ancestry_subset == anc
        change_data.append(mean_change_per_patient[mask])
        change_labels.append(f"{anc}\n(n={stats['n']:,})")
    
    ax2.boxplot(change_data, labels=change_labels)
    ax2.set_ylabel('Absolute Change in Theta', fontsize=12, fontweight='bold')
    ax2.set_title('Distribution of Changes by Ancestry\n(Top 5)', 
                 fontsize=13, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.suptitle('Population Boost: Which Ancestries Show Most Changes with PC Adjustment?', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Population boost comparison complete")
else:
    print("⚠️  Need thetas and ancestry data for population boost comparison")


## Summary

**Key Findings:**
- Phi (signature-disease associations) is highly stable across PC adjustment
- Lambda/Theta (patient signatures) shows high correlation but some differences
- Population changes vary by ancestry - some populations show more changes than others

**Interpretation:**
PC adjustment:
- ✅ Does NOT change biological interpretations (phi stable)
- ⚠️  Changes individual patient signatures (lambda/theta) - this is expected as PCs capture ancestry structure
- 📊 Different ancestries affected differently - expected due to different PC loadings
